In [1]:
import pandas as pd
import glob
import os
from konlpy.tag import Okt
import re

# 이모지 제거 함수 정의
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # 이모티콘
        "\U0001F300-\U0001F5FF"  # 기호 및 상징
        "\U0001F680-\U0001F6FF"  # 교통 및 지도 기호
        "\U0001F700-\U0001F77F"  # 추가 기호 및 상징
        "\U0001F780-\U0001F7FF"  # 기호
        "\U0001F800-\U0001F8FF"  # 추가 상징
        "\U0001F900-\U0001F9FF"  # 추가 이모지
        "\U0001FA00-\U0001FA6F"  # 추가 교통 기호 및 상징
        "\U0001FA70-\U0001FAFF"  # 추가 기호
        "\U00002702-\U000027B0"  # 기타 기호
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# 엑셀 파일들이 저장된 폴더 경로 지정
folder_path = r'C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전'
file_pattern = os.path.join(folder_path, '*.xlsx')

# 형태소 분석기 초기화
okt = Okt()

# 모든 엑셀 파일을 가져오기 (임시 파일 제외)
excel_files = [file for file in glob.glob(file_pattern) if not os.path.basename(file).startswith('~$')]

# 각 엑셀 파일 처리
for file in excel_files:
    # 엑셀 파일 읽어오기 (첫 번째 행을 컬럼으로 사용하지 않음)
    df = pd.read_excel(file, header=None)
    df.reset_index(drop=True, inplace=True)  # 인덱스 리셋하여 불필요한 인덱스 불일치 방지
    
    # 댓글 내용이 있는 컬럼 인덱스를 직접 지정 (1번 컬럼)
    cleaned_comments = []
    
    for comment in df[1].dropna():
        # 이모지 제거
        comment = remove_emoji(comment)
        # 1. 유튜브 타임라인 형식 제거
        comment = re.sub(r'\b\d{1,2}:\d{2}\b', '', comment)
        # 2. 특수 문자 및 기호 제거
        comment = re.sub(r'[^\w\s]', '', comment)
        
        # 형태소 분석으로 단어 분리
        words = okt.morphs(comment)
        filtered_words = [
            word for word in words 
            if not any(char in 'ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅏㅑㅓㅕㅗㅛㅜㅠㅡㅣ' for char in word)  # 감탄사나 단독 자음/모음이 포함된 단어 제거
        ]
        # 단어들을 다시 결합
        cleaned_comment = ' '.join(filtered_words)
        cleaned_comments.append(cleaned_comment)
    
    # 기존 데이터프레임에 수정된 댓글 내용 추가
    df[1] = cleaned_comments + [''] * (len(df) - len(cleaned_comments))  # 원래 길이와 맞춤
    
    # 빈 문자열만 남은 행 제거
    df_filtered = df[df[1] != '']

    # 수정된 데이터프레임을 저장
    output_path = os.path.splitext(file)[0] + '_1.xlsx'
    df_filtered.to_excel(output_path, index=False, header=False)
    print(f"{output_path} 파일로 저장 완료")

print("모든 파일 처리가 완료되었습니다.")


C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\네멋대로해라_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\명수는 12살_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\무인도 특집_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\무한상사 면접편_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\무한상사 야유회_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\방콕 특집_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\서해안고속도로가요제_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\인생극장 특집_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\죄와 길_1.xlsx 파일로 저장 완료
C:\Users\kdp\Desktop\KDW\EX_자연어처리\data\무한도전\짝꿍 특집_1.xlsx 파일로 저장 완료
모든 파일 처리가 완료되었습니다.
